# Y~G ACGT Probabilities 1d (resnet)
> 

In [ ]:
import numpy as np
import pandas as pd

from EnvDL.core import ensure_dir_path_exists 
from EnvDL.dlfn import g2fc_datawrapper, BigDataset, plDNN_general
from EnvDL.dlfn import ResNet1d, BasicBlock1d

import torch
import torch.nn.functional as F # F.mse_loss
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch import nn

import lightning.pytorch as pl
from lightning.pytorch.loggers import TensorBoardLogger

In [ ]:
# Run settings: 
max_epoch = 20
batch_size = 24

In [ ]:
use_gpu_num = 0

device = "cuda" if torch.cuda.is_available() else "cpu"
if use_gpu_num in [0, 1]: 
    torch.cuda.set_device(use_gpu_num)
print(f"Using {device} device")

Using cuda device


In [ ]:
cache_path = '../nbs_artifacts/02.23_g2fc_G_ACGT_conv1d_resnet/'
save_prefix = [e for e in cache_path.split('/') if e != ''][-1]
ensure_dir_path_exists(dir_path = cache_path)

## Load data

In [ ]:
X = g2fc_datawrapper()
X.set_split()
X.load_all(name_list = ['obs_geno_lookup', 'YMat', 'ACGT',], store=True) 

X.calc_cs('YMat', version = 'np', filter = 'val:train')

Loading and storing default `phno`.


In [ ]:
training_dataloader = DataLoader(BigDataset(
    lookups_are_filtered = False,
    lookup_obs  = X.get('val:train',       ops_string='   asarray from_numpy      '),
    lookup_geno = X.get('obs_geno_lookup', ops_string='   asarray from_numpy      '),
    y =           X.get('YMat',            ops_string='cs asarray from_numpy float')[:, None],
    G =           X.get('ACGT',            ops_string='           from_numpy float'),
    G_type = 'raw',
    send_batch_to_gpu = 'cuda:0'
    ),
    batch_size = batch_size,
    shuffle = True
)


validation_dataloader = DataLoader(BigDataset(
    lookups_are_filtered = False,
    lookup_obs = X.get('val:test',         ops_string='   asarray from_numpy      '),
    lookup_geno = X.get('obs_geno_lookup', ops_string='   asarray from_numpy      '),
    y =          X.get('YMat',             ops_string='cs asarray from_numpy float')[:, None],
    G =          X.get('ACGT',             ops_string='           from_numpy float'),
    G_type = 'raw',
    send_batch_to_gpu = 'cuda:0'
    ),
    batch_size = batch_size,
    shuffle = False
)

## Test Models

In [ ]:
model = ResNet1d(
        block = BasicBlock1d, #: Type[Union[BasicBlock, Bottleneck]],
        layers = [2, 2, 2, 2], #: List[int],
        # num_classes: int = 1000,
        zero_init_residual = False,
        groups = 1,
        width_per_group = 64,
        replace_stride_with_dilation = None,
        norm_layer = None,
        input_channels = 4
    ).to('cuda')


DNNG = plDNN_general(model)     
optimizer = DNNG.configure_optimizers()

logger = TensorBoardLogger("tb_logs", name=save_prefix+'-res-4rep2')
trainer = pl.Trainer(max_epochs=max_epoch, logger=logger)

trainer.fit(model=DNNG, train_dataloaders=training_dataloader, val_dataloaders=validation_dataloader)

/home/kickd/miniconda3/envs/fastai/lib/python3.11/site-packages/lightning/fabric/plugins/environments/slurm.py:191: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /home/kickd/miniconda3/envs/fastai/lib/python3.11/si ...
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA GeForce RTX 4090') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
Missing logger folder: tb_logs/g-cnn1-res-4rep2-from-pytorch
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name | Type     | Params

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/home/kickd/miniconda3/envs/fastai/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:441: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=31` in the `DataLoader` to improve performance.
/home/kickd/miniconda3/envs/fastai/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:441: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=31` in the `DataLoader` to improve performance.


Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

/home/kickd/miniconda3/envs/fastai/lib/python3.11/site-packages/lightning/pytorch/trainer/call.py:54: Detected KeyboardInterrupt, attempting graceful shutdown...


In [ ]:
import time
save_time = time.strftime("%Y-%m-%d-%H-%M-%S", time.localtime())

pt_path = cache_path+''.join([save_prefix,'__'+save_time+'-res-4rep2','.pt'])

torch.save(DNNG.mod, pt_path)


In [ ]:
model = ResNet1d(
        block = BasicBlock1d, #: Type[Union[BasicBlock, Bottleneck]],
        layers = [4, 4, 4, 4], #: List[int],
        # num_classes: int = 1000,
        zero_init_residual = False,
        groups = 1,
        width_per_group = 64,
        replace_stride_with_dilation = None,
        norm_layer = None,
        input_channels = 4
    ).to('cuda')


DNNG = plDNN_general(model)     
optimizer = DNNG.configure_optimizers()

logger = TensorBoardLogger("tb_ACGT_logs", name=save_prefix+'-res-4rep4')
trainer = pl.Trainer(max_epochs=max_epoch, logger=logger)

trainer.fit(model=DNNG, train_dataloaders=training_dataloader, val_dataloaders=validation_dataloader)
# 3 hours to train with max_epoch = 20, batch_size = 24

/home/kickd/miniconda3/envs/fastai/lib/python3.11/site-packages/lightning/fabric/plugins/environments/slurm.py:191: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /home/kickd/miniconda3/envs/fastai/lib/python3.11/si ...
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA GeForce RTX 4090') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name | Type     | Params
----------------------------------
0 | mod  | ResNet1d | 8.0

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/home/kickd/miniconda3/envs/fastai/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:441: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=31` in the `DataLoader` to improve performance.
/home/kickd/miniconda3/envs/fastai/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:441: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=31` in the `DataLoader` to improve performance.


Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

In [ ]:
save_time = time.strftime("%Y-%m-%d-%H-%M-%S", time.localtime())

pt_path = cache_path+''.join([save_prefix,'__'+save_time+'-res-4rep4','.pt'])

torch.save(DNNG.mod, pt_path)

In [ ]:
# # torch.save(DNNG.mod, cache_path+'g-acgt-hilb'+'.pt')
# import time, json
# save_time = time.strftime("%Y-%m-%d-%H-%M-%S", time.localtime())

# pt_path = cache_path+''.join([save_prefix,'__'+save_time,'.pt'])

# torch.save(DNNG.mod, pt_path)

3.216666666666667